# Quickstart: Function Calling and Tools

This notebook demonstrates how to use function calling (tools) with v-router across different LLM providers.

## What is Function Calling?

Function calling allows LLMs to interact with external tools and APIs by generating structured function calls. v-router provides:
- **Unified tool interface** across all providers (Anthropic, OpenAI, Google, Azure)
- **Automatic tool inheritance** for fallback models
- **Type-safe tool definitions** using Pydantic schemas
- **Consistent response format** for tool calls across providers

## Core Tool Components

### Request Models
- **`ToolCall`**: Definition of a single tool with name, description, and input schema
- **`Tools`**: Collection of multiple tools to pass to the LLM
- **`LLM`**: Configuration that includes tools alongside model parameters

### Response Models
- **`Response.tool_use`**: List of tool calls made by the model
- **`ToolUse`**: Individual tool call with name, ID, and parsed arguments
- **`Response.content`**: Text content alongside tool calls
- **Unified format**: Same structure across all providers

## Basic Function Calling

Let's start with a simple weather tool example to demonstrate the fundamentals.

In [1]:
from pydantic import BaseModel, Field
from v_router import Client, LLM
from v_router.classes.tools import ToolCall, Tools

# Define the tool schema using Pydantic
class WeatherQuery(BaseModel):
    """Schema for weather query parameters."""
    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")
    units: str = Field("fahrenheit", description="Temperature units: fahrenheit or celsius")

# Create a tool definition
weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Create tools collection
tools = Tools(tools=[weather_tool])

# Create LLM configuration with tools
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=tools  # Pass the tools to the LLM
)

client = Client(llm_config)

# Make a request that should trigger the tool
response = await client.messages.create(
    messages=[
        {"role": "user", "content": "What's the weather like in San Francisco?"}
    ]
)

print(f"Response model: {response.model}")
print(f"Provider: {response.provider}")

# Check the unified response format
print(f"\nContent blocks ({len(response.content)}):")
for i, content in enumerate(response.content):
    print(f"  Block {i}: type={content.type}, role={content.role}")
    if content.type == "text":
        print(f"    Text: {content.text}")

print(f"\nTool uses ({len(response.tool_use)}):")
for i, tool_use in enumerate(response.tool_use):
    print(f"  Tool {i}: {tool_use.name}")
    print(f"    ID: {tool_use.id}")
    print(f"    Arguments: {tool_use.arguments}")

print(f"\nUsage:")
print(f"  Input tokens: {response.usage.input_tokens}")
print(f"  Output tokens: {response.usage.output_tokens}")

2025-05-30 14:46:44,266 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


Response model: claude-sonnet-4-20250514
Provider: anthropic

Content blocks (1):
  Block 0: type=text, role=assistant
    Text: I'll check the current weather in San Francisco for you.

Tool uses (1):
  Tool 0: get_weather
    ID: toolu_01MXmsxvqXm4mLuMN2cA55Kr
    Arguments: {'location': 'San Francisco, CA'}

Usage:
  Input tokens: 462
  Output tokens: 69


## Multiple Tools Example

You can define multiple tools for more complex workflows. Each tool is defined with its own Pydantic schema for type safety.

In [2]:
# Define multiple tool schemas
class CalculatorQuery(BaseModel):
    """Schema for calculator operations."""
    operation: str = Field(..., description="The mathematical operation: add, subtract, multiply, divide")
    a: float = Field(..., description="First number")
    b: float = Field(..., description="Second number")

class TimeQuery(BaseModel):
    """Schema for time queries."""
    timezone: str = Field("UTC", description="Timezone (e.g., UTC, America/New_York, Europe/London)")

# Create multiple tools
calculator_tool = ToolCall(
    name="calculator",
    description="Perform basic mathematical operations",
    input_schema=CalculatorQuery.model_json_schema()
)

time_tool = ToolCall(
    name="get_current_time",
    description="Get the current time in a specified timezone",
    input_schema=TimeQuery.model_json_schema()
)

weather_tool = ToolCall(
    name="get_weather",
    description="Get the current weather in a given location",
    input_schema=WeatherQuery.model_json_schema()
)

# Create tools collection with multiple tools
multi_tools = Tools(tools=[calculator_tool, time_tool, weather_tool])

# Create LLM configuration with multiple tools
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=multi_tools
)

client = Client(llm_config)

# Test with a query that could use multiple tools
response = await client.messages.create(
    messages=[
        {"role": "user", "content": "What's 15 * 23? Also, what time is it in New York?"}
    ]
)

print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

# Show the unified response format
print(f"\n📝 Content blocks: {len(response.content)}")
for content in response.content:
    print(f"  - {content.type}: {content.text[:50]}..." if len(content.text) > 50 else f"  - {content.type}: {content.text}")

print(f"\n🔧 Tool uses: {len(response.tool_use)}")
for tool in response.tool_use:
    print(f"  - {tool.name}: {tool.arguments}")

2025-05-30 14:46:46,863 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


Provider: anthropic
Model: claude-sonnet-4-20250514

📝 Content blocks: 1
  - text: I'll help you with both of those questions!

🔧 Tool uses: 2
  - calculator: {'operation': 'multiply', 'a': 15, 'b': 23}
  - get_current_time: {'timezone': 'America/New_York'}


## Cross-Provider Function Calling

The same tools work across different providers! This demonstrates v-router's unified interface.

In [3]:
# Same tools, different provider - OpenAI
llm_config_openai = LLM(
    model_name="gpt-4",
    provider="openai",
    tools=multi_tools  # Same tools work across providers!
)

client_openai = Client(llm_config_openai)

response_openai = await client_openai.messages.create(
    messages=[
        {"role": "user", "content": "Calculate 42 divided by 7"}
    ]
)

print("=== OpenAI Provider ===")
print(f"Provider: {response_openai.provider}")
print(f"Model: {response_openai.model}")

# Unified response format across all providers!
print(f"\n📝 Content: {len(response_openai.content)} blocks")
for content in response_openai.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

print(f"\n🔧 Tool uses: {len(response_openai.tool_use)}")
for tool_use in response_openai.tool_use:
    print(f"  Tool: {tool_use.name}")
    print(f"  Arguments: {tool_use.arguments}")

# Let's also try with Google
llm_config_google = LLM(
    model_name="gemini-1.5-pro",
    provider="google",
    tools=Tools(tools=[calculator_tool])  # Just the calculator for Google
)

client_google = Client(llm_config_google)

response_google = await client_google.messages.create(
    messages=[
        {"role": "user", "content": "What is 8 times 9?"}
    ]
)

print("\n=== Google Provider ===")
print(f"Provider: {response_google.provider}")
print(f"Model: {response_google.model}")

# Same unified format for Google!
print(f"\n📝 Content: {len(response_google.content)} blocks")
for content in response_google.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

print(f"\n🔧 Tool uses: {len(response_google.tool_use)}")
for tool_use in response_google.tool_use:
    print(f"  Tool: {tool_use.name}")
    print(f"  Arguments: {tool_use.arguments}")

print("\n✅ Notice how all providers now return the same Response format!")

2025-05-30 14:46:51,067 - v_router.router - INFO - Trying primary model: gpt-4 on openai
2025-05-30 14:46:52,999 - v_router.router - INFO - Trying primary model: gemini-1.5-pro on google


=== OpenAI Provider ===
Provider: openai
Model: gpt-4-0613

📝 Content: 0 blocks

🔧 Tool uses: 1
  Tool: calculator
  Arguments: {'operation': 'divide', 'a': 42, 'b': 7}

=== Google Provider ===
Provider: google
Model: gemini-1.5-pro

📝 Content: 0 blocks

🔧 Tool uses: 1
  Tool: calculator
  Arguments: {'a': 8, 'operation': 'multiply', 'b': 9}

✅ Notice how all providers now return the same Response format!


## Function Calling with Fallbacks

Tools work seamlessly with the fallback system. If the primary model fails, the backup models will automatically inherit the same tools.

### How Tool Inheritance Works:
1. **Primary Model**: Defines tools in the main LLM configuration
2. **Automatic Inheritance**: Backup models inherit tools from primary (no need to redefine)
3. **Provider Translation**: Tools are automatically translated to each provider's format
4. **Seamless Fallback**: If primary fails, backup models have the same capabilities

In [4]:
from v_router import BackupModel

# Create a configuration with a non-existent primary model and fallbacks
llm_config_with_fallback = LLM(
    model_name="claude-nonexistent",  # This will fail
    provider="anthropic",
    tools=multi_tools,  # Tools will be passed to fallback models too
    backup_models=[
        BackupModel(
            model=LLM(
                model_name="gpt-4",
                provider="openai"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=1
        ),
        BackupModel(
            model=LLM(
                model_name="gemini-1.5-pro", 
                provider="google"
                # Note: No tools specified here - they'll be inherited from primary
            ),
            priority=2
        )
    ]
)

client_fallback = Client(llm_config_with_fallback)

# This will fail on the primary model but succeed on the fallback with tools intact
response_fallback = await client_fallback.messages.create(
    messages=[
        {"role": "user", "content": "Please calculate 100 divided by 5"}
    ]
)

print(f"Provider used: {response_fallback.provider}")
print(f"Model used: {response_fallback.model}")

# Check if tools were used (unified format)
print(f"\n✅ Tools work seamlessly with fallbacks!")
print(f"Content blocks: {len(response_fallback.content)}")
for content in response_fallback.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

if response_fallback.tool_use:
    print(f"\nTool calls: {len(response_fallback.tool_use)}")
    for tool_use in response_fallback.tool_use:
        print(f"  Tool: {tool_use.name}")
        print(f"  Arguments: {tool_use.arguments}")
        
print(f"\n💡 The backup model inherited tools from the primary configuration!")

2025-05-30 14:46:53,995 - v_router.router - INFO - Trying primary model: claude-nonexistent on anthropic
2025-05-30 14:46:54,218 - v_router.router - WARNING - Primary model failed: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-nonexistent'}}
2025-05-30 14:46:54,219 - v_router.router - INFO - Trying backup model: gpt-4 on openai


Provider used: openai
Model used: gpt-4-0613

✅ Tools work seamlessly with fallbacks!
Content blocks: 0

Tool calls: 1
  Tool: calculator
  Arguments: {'operation': 'divide', 'a': 100, 'b': 5}

💡 The backup model inherited tools from the primary configuration!


## Handling Tool Responses

In a real application, you'd want to execute the tools and provide results back to the model. Here's a complete example of the full tool calling workflow.

In [5]:
import json
from datetime import datetime

def execute_tool(tool_name: str, tool_input: dict) -> str:
    """Simulate executing tools and returning results."""
    if tool_name == "calculator":
        operation = tool_input["operation"]
        a = tool_input["a"]
        b = tool_input["b"]
        
        if operation == "add":
            result = a + b
        elif operation == "subtract":
            result = a - b
        elif operation == "multiply":
            result = a * b
        elif operation == "divide":
            result = a / b if b != 0 else "Error: Division by zero"
        else:
            result = "Error: Unknown operation"
            
        return f"The result of {a} {operation} {b} is {result}"
    
    elif tool_name == "get_current_time":
        # Simulate getting current time
        return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (simulated)"
    
    elif tool_name == "get_weather":
        location = tool_input["location"]
        units = tool_input.get("units", "fahrenheit")
        # Simulate weather data
        return f"The weather in {location} is 72°F (22°C), sunny with light winds (simulated)"
    
    return "Tool execution not implemented"

# Create a simple calculator tool for this example
simple_calc_tool = ToolCall(
    name="calculator", 
    description="Perform basic math operations",
    input_schema=CalculatorQuery.model_json_schema()
)

llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=Tools(tools=[simple_calc_tool])
)

client = Client(llm_config)

# Initial request
messages = [
    {"role": "user", "content": "What is 25 times 4?"}
]

response = await client.messages.create(messages=messages)

print("Initial response:")
print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

# Display content blocks
print(f"\nContent blocks: {len(response.content)}")
for content in response.content:
    if content.type == "text":
        print(f"  Text: {content.text}")

# Check if there are tool calls in the unified format
if response.tool_use:
    print(f"\nFound {len(response.tool_use)} tool call(s):")
    
    # For Anthropic, we need to build the proper message format
    # We'll use the raw_response for the exact format Anthropic expects
    messages.append({
        "role": "assistant", 
        "content": response.raw_response["content"] if isinstance(response.raw_response, dict) else response.raw_response.content
    })
    
    # Execute each tool and add results
    for tool_use in response.tool_use:
        print(f"Executing tool: {tool_use.name}")
        print(f"Input: {tool_use.arguments}")
        
        # Execute the tool
        result = execute_tool(tool_use.name, tool_use.arguments)
        print(f"Result: {result}")
        
        # Add tool result to conversation
        messages.append({
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": result
                }
            ]
        })
    
    # Get final response with tool results
    final_response = await client.messages.create(messages=messages)
    print(f"\nFinal response with tool results:")
    print(f"Content blocks: {len(final_response.content)}")
    for content in final_response.content:
        if content.type == "text":
            print(f"  Text: {content.text}")
else:
    print("\nNo tool calls found in response")

2025-05-30 14:46:56,344 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic
2025-05-30 14:46:59,554 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


Initial response:
Provider: anthropic
Model: claude-sonnet-4-20250514

Content blocks: 1
  Text: I'll calculate 25 times 4 for you.

Found 1 tool call(s):
Executing tool: calculator
Input: {'operation': 'multiply', 'a': 25, 'b': 4}
Result: The result of 25 multiply 4 is 100

Final response with tool results:
Content blocks: 1
  Text: 25 times 4 equals 100.


## Advanced Tool Patterns

Let's explore some advanced patterns for tool usage:

In [6]:
# Complex tool with nested data structures
from typing import List, Optional

class SearchResult(BaseModel):
    """Individual search result."""
    title: str
    url: str
    snippet: str

class WebSearchQuery(BaseModel):
    """Schema for web search operations."""
    query: str = Field(..., description="Search query string")
    max_results: int = Field(5, description="Maximum number of results to return")
    include_snippets: bool = Field(True, description="Whether to include content snippets")
    domains: Optional[List[str]] = Field(None, description="Specific domains to search within")

class DataAnalysisQuery(BaseModel):
    """Schema for data analysis operations."""
    data_source: str = Field(..., description="Path or identifier for the data source")
    analysis_type: str = Field(..., description="Type of analysis: summary, correlation, trend, distribution")
    columns: Optional[List[str]] = Field(None, description="Specific columns to analyze")
    filters: Optional[dict] = Field(None, description="Filters to apply to the data")

# Create advanced tools
web_search_tool = ToolCall(
    name="web_search",
    description="Search the web for information and return relevant results",
    input_schema=WebSearchQuery.model_json_schema()
)

data_analysis_tool = ToolCall(
    name="analyze_data",
    description="Perform statistical analysis on datasets",
    input_schema=DataAnalysisQuery.model_json_schema()
)

# Combine with existing tools
advanced_tools = Tools(tools=[
    calculator_tool,
    weather_tool, 
    web_search_tool,
    data_analysis_tool
])

# Test with complex query
llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=advanced_tools
)

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {
            "role": "user", 
            "content": "I need to research machine learning trends. Can you search for recent ML developments and also calculate what 15% of 2000 is?"
        }
    ]
)

print("=== Advanced Tool Usage ===")
print(f"Provider: {response.provider}")
print(f"Model: {response.model}")

print(f"\n📝 Content: {len(response.content)} blocks")
for content in response.content:
    if content.type == "text":
        print(f"  Text: {content.text[:100]}..." if len(content.text) > 100 else f"  Text: {content.text}")

print(f"\n🔧 Tool calls: {len(response.tool_use)}")
for tool_use in response.tool_use:
    print(f"  Tool: {tool_use.name}")
    print(f"  Arguments: {json.dumps(tool_use.arguments, indent=2)}")

print(f"\n💡 Complex tools with nested schemas work seamlessly!")

2025-05-30 14:47:01,126 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


=== Advanced Tool Usage ===
Provider: anthropic
Model: claude-sonnet-4-20250514

📝 Content: 1 blocks
  Text: I'll help you with both tasks - searching for recent machine learning developments and calculating 1...

🔧 Tool calls: 2
  Tool: web_search
  Arguments: {
  "query": "recent machine learning trends developments 2024",
  "max_results": 5
}
  Tool: calculator
  Arguments: {
  "operation": "multiply",
  "a": 2000,
  "b": 0.15
}

💡 Complex tools with nested schemas work seamlessly!


## Understanding the Tool Response Format

Let's examine the unified tool response format in detail:

In [7]:
# Simple tool call to examine response structure
simple_tools = Tools(tools=[calculator_tool])

llm_config = LLM(
    model_name="claude-sonnet-4-20250514",
    provider="anthropic",
    tools=simple_tools
)

client = Client(llm_config)

response = await client.messages.create(
    messages=[
        {"role": "user", "content": "Calculate 7 * 8"}
    ]
)

print("🔍 Detailed Response Structure Analysis:")
print(f"\n1. Basic Response Info:")
print(f"   ├── Provider: {response.provider}")
print(f"   ├── Model: {response.model}")
print(f"   └── Response Type: {type(response).__name__}")

print(f"\n2. Content Blocks ({len(response.content)} total):")
for i, content in enumerate(response.content):
    print(f"   Block {i}:")
    print(f"   ├── Type: {content.type}")
    print(f"   ├── Role: {content.role}")
    print(f"   └── Text: {content.text[:50]}..." if len(content.text) > 50 else f"   └── Text: {content.text}")

print(f"\n3. Tool Use Blocks ({len(response.tool_use)} total):")
for i, tool_use in enumerate(response.tool_use):
    print(f"   Tool {i}:")
    print(f"   ├── Name: {tool_use.name}")
    print(f"   ├── ID: {tool_use.id}")
    print(f"   ├── Arguments Type: {type(tool_use.arguments).__name__}")
    print(f"   └── Arguments: {tool_use.arguments}")

print(f"\n4. Usage Information:")
print(f"   ├── Input Tokens: {response.usage.input_tokens}")
print(f"   ├── Output Tokens: {response.usage.output_tokens}")
print(f"   └── Total Tokens: {response.usage.input_tokens + response.usage.output_tokens}")

print(f"\n5. Raw Response:")
print(f"   ├── Type: {type(response.raw_response).__name__}")
print(f"   └── Available for provider-specific processing")

print("\n✅ This exact structure works across ALL providers!")

2025-05-30 14:47:05,452 - v_router.router - INFO - Trying primary model: claude-sonnet-4-20250514 on anthropic


🔍 Detailed Response Structure Analysis:

1. Basic Response Info:
   ├── Provider: anthropic
   ├── Model: claude-sonnet-4-20250514
   └── Response Type: Response

2. Content Blocks (1 total):
   Block 0:
   ├── Type: text
   ├── Role: assistant
   └── Text: I'll calculate 7 * 8 for you.

3. Tool Use Blocks (1 total):
   Tool 0:
   ├── Name: calculator
   ├── ID: toolu_01S9LR75phscGEWh64PrEhNE
   ├── Arguments Type: dict
   └── Arguments: {'operation': 'multiply', 'a': 7, 'b': 8}

4. Usage Information:
   ├── Input Tokens: 470
   ├── Output Tokens: 99
   └── Total Tokens: 569

5. Raw Response:
   ├── Type: dict
   └── Available for provider-specific processing

✅ This exact structure works across ALL providers!


## Summary

### Key Features Demonstrated:

✅ **Unified Tool Interface**: Same tool definitions work across Anthropic, OpenAI, Google, and Azure  
✅ **Type-Safe Schemas**: Use Pydantic models for robust tool parameter validation  
✅ **Automatic Tool Inheritance**: Backup models inherit tools from primary configuration  
✅ **Consistent Response Format**: Same tool response structure across all providers  
✅ **Complex Tool Support**: Handle nested data structures and multiple parameters  
✅ **Full Conversation Flow**: Execute tools and continue conversations with results  

### Tool Request Models:
- **`ToolCall`**: Individual tool definition (name, description, schema)
- **`Tools`**: Collection of tools to pass to LLM
- **`LLM.tools`**: Tools configuration in LLM setup
- **Pydantic Models**: Type-safe parameter schemas

### Tool Response Models:
- **`Response.tool_use`**: List of tool calls made by the model
- **`ToolUse`**: Individual tool call with name, ID, and arguments
- **`Response.content`**: Text content alongside tool calls
- **`Response.usage`**: Token usage including tool call tokens

### Advanced Patterns:
- **Multi-step Workflows**: Chain tool calls together
- **Complex Schemas**: Use nested data structures and optional parameters
- **Provider Fallback**: Tools work seamlessly with backup models
- **Tool Result Handling**: Full conversation flow with tool execution

### Next Steps:
- Check out `quickstart_models.ipynb` for basic model usage and fallbacks
- Explore custom tool implementations for your specific use cases
- See the full documentation for advanced tool patterns

v-router provides the most comprehensive and unified function calling interface across all major LLM providers!